In [ ]:
# !pip install selenium

In [ ]:
# !pip install webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By

In [2]:
import pandas as pd

In [3]:
faq_df = pd.DataFrame(columns=["source", "question", "answer"])
faq_df

,source,question,answer


In [4]:
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
options.add_argument("--disable-gpu")

In [5]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [6]:
# driver.get("https://odsgm.meb.gov.tr/www/sss.php")
url = "https://www.yok.gov.tr/iletisim/sss"
driver.get(url)

print(driver.title)

YÖK sss


In [7]:
current_page = driver.find_element(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/span[1]')
next_page = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a[text() = "2"]')

current_page_number = current_page.text
current_page_number

'1'

In [8]:
# page_numbers = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a')
# page_numbers = [i.text for i in page_numbers if i!='...']
# page_numbers

In [9]:
# //body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a[text() = '2']

In [10]:
def find_more_button():
    more_button_alt1 = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a[5]')
    more_button_alt2 = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a[6]')
    
    more_button = None

    if (len(more_button_alt1)>0 and more_button_alt1[0].text == "...") or (len(more_button_alt2)>0 and more_button_alt2[0].text == "...") :
        
        if len(more_button_alt2)>0:
            more_button = more_button_alt2[0]
            print(2)
        else:
            more_button = more_button_alt1[0]
            print(1)
        
        # print(more_button.text)
    
    return more_button

In [11]:
import time

In [12]:
num_pages = 38

while(True):

    texts = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div')
    texts = [i.text for i in texts]
    a = list(range(len(texts)))
    ques_idx = list(filter(lambda x: x % 2 == 0, a))
    answ_idx = list(filter(lambda x: x % 2 == 1, a))
    ques = [texts[i].replace('\n', ' ') for i in ques_idx]
    answ = [texts[i].replace('\n', '') for i in answ_idx]
    
    print(ques, answ)
    
    current_page = driver.find_element(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/span[1]')
    current_page_number = int(current_page.text)
    
    srcs = [f"https://www.yok.gov.tr/iletisim/sss/{current_page_number}/{i}" for i in range(1, (len(ques)+1))]

    data_dict = {"source": srcs, "question": ques, "answer":answ}
    
    temp_df = pd.DataFrame(columns=["source", "question", "answer"])
    
    faq_df = pd.concat([faq_df, pd.DataFrame(data=data_dict)])
    
    print(current_page_number)
    
    if current_page_number == num_pages:
        break
    
    if current_page_number%5!=0:
        next_page = driver.find_elements(By.XPATH, f'//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/a[text() = "{current_page_number+1}"]')
        print(next_page[0].click())
    else:
        more_button = find_more_button()
        print(more_button.click())


# more_button = find_more_button()
# while(more_button):
    
#     # questions = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div/h1[1]')
#     answers = driver.find_elements(By.XPATH, '//body[1]/form[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/span[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div')


#     # questions[-2].text
#     print(answers[-1].text)
    
#     more_button.click()
    
#     more_button = find_more_button()
    
#     time.sleep(1)

["ÖSYS'de özürlü öğrenci kontenjanı var mıdır?", 'Meslek Yüksekokulu öğrencileri programdan faydalanabilecekler mi?', "Bologna Eşgüdüm Komisyonu ile ilgili hangi bilgileri YÖK'e göndermemiz gerekiyor?", "Türkiye Bologna Sürecine üye olduğu için, bir T.C vatandaşının Avrupa'da Bologna Sürecine üye diğer ülkelerdeki Yükseköğretim kurumlarından aldığı diplomayı YÖK'ün doğrudan tanıması gerekmez mi?", 'Erasmus programıyla yurtdışına gitmek istiyorum. Sizden bu konuda bilgi alabilir miyim?', 'Üniversitemizdeki BEK (Bologna Eşgüdüm Komisyonu) Komisyonu üyesi değişti. Yeni üyeyi nereye bildirelim?', "Avrupa Üniversiteler Birliği (EUA)'ne üye olmak istiyoruz. Bu konuda bize yardımcı olabilir misiniz?", 'Yabancı dil eşdeğerlilik belgemi onaylıyor musunuz?', 'Başvurular şahsimi üniversite aracılığıylamı ?', "YÖK'teki işlemlerim ne kadar sürüyor?"] ['Yurdumuzda bulunan yükseköğretim kurumlarına giriş, 2547 sayılı Yükseköğretim Kanununun 2880 sayılı Kanunla değişik 45. maddesinde belirtilen ve Öğr

In [13]:
faq_df

,source,question,answer
0,https://www.yok.gov.tr/iletisim/sss/1/1,ÖSYS'de özürlü öğrenci kontenjanı var mıdır?,Yurdumuzda bulunan yükseköğretim kurumlarına g...
1,https://www.yok.gov.tr/iletisim/sss/1/2,Meslek Yüksekokulu öğrencileri programdan fayd...,Evet
2,https://www.yok.gov.tr/iletisim/sss/1/3,Bologna Eşgüdüm Komisyonu ile ilgili hangi bil...,Üniversitenizin Bologna Eşgüdüm Komisyonu başk...
3,https://www.yok.gov.tr/iletisim/sss/1/4,"Türkiye Bologna Sürecine üye olduğu için, bir ...",Hayır. Bologna Süreci doğrudan tanınmayı amaçl...
4,https://www.yok.gov.tr/iletisim/sss/1/5,Erasmus programıyla yurtdışına gitmek istiyoru...,Erasmus'un da dahil olduğu Avrupa Birliği Eğit...
...,...,...,...
6,https://www.yok.gov.tr/iletisim/sss/37/7,Mevlana Değişim Programı Protokolü’ne ek olara...,Anlaşma yapılacak yükseköğretim kurumlarıyla ö...
7,https://www.yok.gov.tr/iletisim/sss/37/8,Ek protokol için standart bir format var mı? E...,Ek protokol için standart bir format ve içeriğ...
8,https://www.yok.gov.tr/iletisim/sss/37/9,Mevlana Değişim Programı Protokolü’nü Rektörün...,Rektörün bulunmadığı durumlarda Protokolü sade...
9,https://www.yok.gov.tr/iletisim/sss/37/10,İmzaladığımız Protokollerin bir kopyasını YÖK’...,"İmzalanan Protokolün elektronik kopyası, fotok..."


In [14]:
# faq_df["question"] = faq_df["question"].replace('\n', ' ')
# faq_df["answer"] = faq_df["answer"].replace('\n', ' ')

In [15]:
faq_df.to_csv("yok_faq.csv", index=False)

In [ ]:
driver.close()